#<center>**Agrupamento e PCA**</center>

<b>Autora:</b> Maria Eline Silva de Farias- 516229

<b>Resumo:</b> A bade de dados é denominada: Shop Customer Data, que é uma análise detalhada dos clientes ideais de uma loja imaginativa. Ajudando uma empresa a entender melhor seus clientes. O dono de uma loja obtém informações sobre os Clientes através de cartões de sócio.
**O conjunto de dados consiste em 2.000 registros e 8 colunas (atributos):**


* Customer ID (Identificação do Cliente)
* Gender (Gênero)
* Age (Idade)
* Annual Income (Rendimento anual)
* Spending Score - Score assigned by the shop, based on customer behavior and spending nature (
Pontuação de gastos - Pontuação atribuída pela loja, com base no comportamento do cliente e na natureza do gasto)
* Profession (Profissão)
* Work Experience - in years (Experiência profissional - em anos)
* Family Size (Tamanho família)



Não usar funções prontas para o K-means e PCA. É permitido
usar uma função existente para o cálculo dos autovalores e
autovetores no PCA.
Você deve **numerizar** os atributos **Gender** e **Profession**.
Fazer agrupamento de dados usando K-means (sozinho) e usando
PCA + K-means. Usar o K-means com a distância Euclidiana.



<b>Datasets URL : </b>

* https://www.kaggle.com/datasets/datascientistanna/customers-dataset

<b>Descrição:</b>

Este projeto é realizado para  a disciplina Tópicos Especiais em Telecomunicações I (Reconhecimento de Padrões).

##Importando Bibliotecas

In [18]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import math
import plotly.express as px
from scipy import stats
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np
import math
from scipy.spatial.distance import cdist

##Carregando a Base de Dados

In [19]:
# carregando dataset inputData
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/trabalho06/dados/Customers.csv",header=None)

In [20]:
#Exibe 10 primeiras linhas do dataset
df.head(10)

,0,1,2,3,4,5,6,7
0,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
1,1,Male,19,15000,39,Healthcare,1,4
2,2,Male,21,35000,81,Engineer,3,3
3,3,Female,20,86000,6,Engineer,1,1
4,4,Female,23,59000,77,Lawyer,0,2
5,5,Female,31,38000,40,Entertainment,2,6
6,6,Female,22,58000,76,Artist,0,2
7,7,Female,35,31000,6,Healthcare,1,3
8,8,Female,23,84000,94,Healthcare,1,3
9,9,Male,64,97000,3,Engineer,0,3


In [21]:
# Definir a primeira linha como o nome das colunas
df.set_axis(df.iloc[0], axis=1, inplace=True)
df = df[1:]
#Exibe 10 primeiras linhas do dataset
df.head(10)

<ipython-input-21-0d01ec5e6500>:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(df.iloc[0], axis=1, inplace=True)


,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
1,1,Male,19,15000,39,Healthcare,1,4
2,2,Male,21,35000,81,Engineer,3,3
3,3,Female,20,86000,6,Engineer,1,1
4,4,Female,23,59000,77,Lawyer,0,2
5,5,Female,31,38000,40,Entertainment,2,6
6,6,Female,22,58000,76,Artist,0,2
7,7,Female,35,31000,6,Healthcare,1,3
8,8,Female,23,84000,94,Healthcare,1,3
9,9,Male,64,97000,3,Engineer,0,3
10,10,Female,30,98000,72,Artist,1,4


##Pré- Processamento dos dados

**Converter os atributos categóricos em valores numéricos:**

* O atributo "Gender" será codificado, com a seguinte atribuição:
 - Male = 0
 - Female = 1

* O atributo "Profession" será codificado, com a seguinte atribuição:
 - Healthcare = 0
 - Engineer = 1
 - Lawyer = 2
 - Entertainment = 3
 - Artist = 4
 - Executive = 5
 - Doctor = 6
 - Homemaker = 7
 - Marketing = 8
 - nan = 9

In [22]:
#Verificando a possivel existencia de valores nulos dentro do dataset
df.isnull().sum()

0
CustomerID                 0
Gender                     0
Age                        0
Annual Income ($)          0
Spending Score (1-100)     0
Profession                35
Work Experience            0
Family Size                0
dtype: int64

In [23]:
#Removendo amostras que possuem valores Nan(Not a Number)
# Remover as linhas com valores NaN
df_sem_nan_linhas = df.dropna()
# atribuindo o df_sem_nan_linhas ao df
df = df_sem_nan_linhas
#detalhes sobre o novo df
df.info()
# Exclua a coluna  0   CustomerID
df = df.drop(df.columns[0], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965 entries, 1 to 2000
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              1965 non-null   object
 1   Gender                  1965 non-null   object
 2   Age                     1965 non-null   object
 3   Annual Income ($)       1965 non-null   object
 4   Spending Score (1-100)  1965 non-null   object
 5   Profession              1965 non-null   object
 6   Work Experience         1965 non-null   object
 7   Family Size             1965 non-null   object
dtypes: object(8)
memory usage: 138.2+ KB


In [24]:
# Atributo Gender
# Mapear os valores de "Gender" para os números correspondentes
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})
# Obter todos os valores únicos da coluna
valores_unicos = df['Gender'].unique()
# Exibir os valores únicos
print(valores_unicos)

[0 1]


In [25]:
# Atributo Profession
# Mapear os valores de "Gender" para os números correspondentes
df['Profession'] = df['Profession'].map({'Healthcare': 0, 'Engineer': 1, 'Lawyer': 2, 'Entertainment': 3,
                                 'Artist': 4, 'Executive': 5, 'Doctor': 6, 'Homemaker': 7, 'Marketing': 8})
# Obter todos os valores únicos da coluna
valores_unicos = df['Profession'].unique()
# Exibir os valores únicos
print(valores_unicos)

[0 1 2 3 4 5 6 7 8]


In [26]:
#Convertendo os outros atributos para o tipo float
atributos_numericos = ['Age', 'Annual Income ($)', 'Spending Score (1-100)', 'Work Experience', 'Family Size']
for atributo in atributos_numericos:
    df[atributo] = df[atributo].astype(float)

In [27]:
#Verificando a possivel existencia de valores nulos dentro do dataset, depois do pré-processamento dos dados
df.isnull().sum()

0
Gender                    0
Age                       0
Annual Income ($)         0
Spending Score (1-100)    0
Profession                0
Work Experience           0
Family Size               0
dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965 entries, 1 to 2000
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Gender                  1965 non-null   int64  
 1   Age                     1965 non-null   float64
 2   Annual Income ($)       1965 non-null   float64
 3   Spending Score (1-100)  1965 non-null   float64
 4   Profession              1965 non-null   int64  
 5   Work Experience         1965 non-null   float64
 6   Family Size             1965 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 122.8 KB


In [29]:
df.head(10)

,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
1,0,19.0,15000.0,39.0,0,1.0,4.0
2,0,21.0,35000.0,81.0,1,3.0,3.0
3,1,20.0,86000.0,6.0,1,1.0,1.0
4,1,23.0,59000.0,77.0,2,0.0,2.0
5,1,31.0,38000.0,40.0,3,2.0,6.0
6,1,22.0,58000.0,76.0,4,0.0,2.0
7,1,35.0,31000.0,6.0,0,1.0,3.0
8,1,23.0,84000.0,94.0,0,1.0,3.0
9,0,64.0,97000.0,3.0,1,0.0,3.0
10,1,30.0,98000.0,72.0,4,1.0,4.0


###Normalizando os dados
O método z-score transforma as informações em distribuição com uma média 0 e um desvio típico de 1.

In [30]:
df_norm = df.copy()
for column in df_norm.columns:
    df_norm[column] = (df_norm[column] - df_norm[column].mean()) / df_norm[column].std()
display(df_norm)

,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
1,-1.207717,-1.052077,-2.086144,-0.431741,-1.490016,-0.787636,0.123327
2,-1.207717,-0.981692,-1.649785,1.069483,-1.035658,-0.278271,-0.384717
3,0.827587,-1.016884,-0.537070,-1.611273,-1.035658,-0.787636,-1.400804
4,0.827587,-0.911306,-1.126155,0.926510,-0.581300,-1.042319,-0.892761
5,0.827587,-0.629763,-1.584331,-0.395997,-0.126943,-0.532954,1.139414
...,...,...,...,...,...,...,...
1996,0.827587,0.777949,1.609530,-0.395997,0.327415,0.995141,1.647457
1997,0.827587,1.481806,-0.817256,-0.681944,1.236131,0.740458,1.647457
1998,-1.207717,1.341034,-0.428832,-1.325326,-1.490016,1.249823,-0.892761
1999,-1.207717,0.989106,1.559829,-1.682760,0.781773,0.740458,-0.892761


In [31]:
df_norm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965 entries, 1 to 2000
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Gender                  1965 non-null   float64
 1   Age                     1965 non-null   float64
 2   Annual Income ($)       1965 non-null   float64
 3   Spending Score (1-100)  1965 non-null   float64
 4   Profession              1965 non-null   float64
 5   Work Experience         1965 non-null   float64
 6   Family Size             1965 non-null   float64
dtypes: float64(7)
memory usage: 122.8 KB


##Agrupamento de dados usando  Somente o K-means

1. Inicialização: Escolher aleatoriamente K centroides iniciais a partir do conjunto de dados.

2. Atribuição de pontos aos grupos: Calcular a distância entre cada ponto e os centroides e atribuir cada ponto ao grupo representado pelo centroide mais próximo.

3. Atualização dos centroides: Calcular os novos centroides para cada grupo, usando a média dos pontos atribuídos a cada grupo.

4. Repetição: Repetir os passos 2 e 3 até que não haja mais mudanças nas atribuições dos pontos aos grupos ou até que um número máximo de iterações seja atingido.

5. Resultados: Retornar os centroides finais e as atribuições dos pontos aos grupos.

- **K = 2: SWC = 0.5317811318974596**
- **K = 3: SWC = 0.5522463209948962**
- **K = 4: SWC = 0.6208341265829714**
- **K = 5: SWC = 0.6055827620966858**


In [32]:
# Função para calcular a distância euclidiana entre dois pontos
def euclidean_distance(ponto1, ponto2):
    #print("ponto1:", ponto1)
    #print("ponto2:", ponto2)
    result = (float(ponto1[0])-float(ponto2[0]))**2 + (float(ponto1[1])-float(ponto2[1]))**2
    return math.sqrt(result)

# Função para inicializar os centróides aleatoriamente
def initialize_centroids(data, k):
    centroids_indices = np.random.choice(range(len(data)), size=k, replace=False)
    centroids = data[centroids_indices]
    return centroids

# Função para atribuir pontos aos clusters
def assign_clusters(data, centroids):
    clusters = []
    for ponto in data:
        distances = [euclidean_distance(ponto, centroid) for centroid in centroids]
        cluster = np.argmin(distances)
        clusters.append(cluster)
    return np.array(clusters)

# Função para atualizar os centróides dos clusters
def update_centroids(data, clusters, k):
    centroids = []
    for cluster in range(k):
        cluster_pontos = data[clusters == cluster]
        centroid = np.mean(cluster_pontos, axis=0)
        centroids.append(centroid)
    return np.array(centroids)

# Função para calcular a largura média de silhueta (SWC)
def calculate_swc(data, clusters):
    swc = 0
    for i, ponto in enumerate(data):
        cluster = clusters[i]
        cluster_pontos = data[clusters == cluster]
        a = np.mean([euclidean_distance(ponto, outro_ponto) for outro_ponto in cluster_pontos if not np.array_equal(ponto, outro_ponto)])

        b = float('inf')
        for outro_cluster in range(len(set(clusters))):
            if outro_cluster != cluster:
                outro_cluster_pontos = data[clusters == outro_cluster]
                b = min(b, np.mean([euclidean_distance(ponto, outro_ponto) for outro_ponto in outro_cluster_pontos]))

        sil_coeff = (b - a) / max(a, b)
        swc += sil_coeff
    return swc / len(data)

# Função principal do K-means
def kmeans(data, k, max_iterations=10):
    melhor_swc = float('-inf')
    melhor_clusters = None

    for _ in range(max_iterations):
        centroids = initialize_centroids(data, k)
        prev_clusters = None

        while True:
            clusters = assign_clusters(data, centroids)
            if np.array_equal(clusters, prev_clusters):
                break

            centroids = update_centroids(data, clusters, k)
            prev_clusters = clusters

        swc = calculate_swc(data, clusters)
        if swc > melhor_swc:
            melhor_swc = swc
            melhor_clusters = clusters

    return melhor_clusters, melhor_swc

# Executando o K-means para diferentes valores de k
k_values = [2, 3, 4, 5]

for k in k_values:
    clusters, swc = kmeans(df_norm.values, k, max_iterations=10)
    print(f"K = {k}: SWC = {swc}")

K = 2: SWC = 0.5317811318974596
K = 3: SWC = 0.5522463209948962
K = 4: SWC = 0.6208341265829714
K = 5: SWC = 0.6055827620966858


##Agrupamento de dados usando PCA com K-means

1. Realizar a redução de dimensionalidade usando PCA para diferentes valores do número de componentes.
2. Aplicar o K-means nos dados transformados pelo PCA.
3. Calcular a Largura Média de Silhueta (SWC) para cada caso e selecionar o melhor resultado.

- **Melhor número de componentes do PCA: 1**
- **Melhor valor de K: 2**
- **SWC: 0.5624557724559919**

In [33]:
def calcular_swc(dados, clusters):
    swc = 0
    for i, ponto in enumerate(dados):
        cluster = clusters[i]
        pontos_cluster = dados[clusters == cluster]

        if len(pontos_cluster) == 0:
            continue

        a = np.mean(cdist([ponto], pontos_cluster, metric='euclidean'))

        b = np.inf
        for outro_cluster in np.unique(clusters):
            if outro_cluster != cluster:
                pontos_outro_cluster = dados[clusters == outro_cluster]
                if len(pontos_outro_cluster) > 0:
                    b = min(b, np.mean(cdist([ponto], pontos_outro_cluster, metric='euclidean')))

        coeficiente_silhueta = (b - a) / max(a, b)
        swc += coeficiente_silhueta

    return swc / len(dados)

def pca(dados, num_componentes):
    # Cálculo da matriz de covariância dos dados
    matriz_covariancia = np.cov(dados.T)

    # Cálculo dos autovalores e autovetores da matriz de covariância
    autovalores, autovetores = np.linalg.eig(matriz_covariancia)

    # Ordenação dos pares autovalor-autovetor em ordem decrescente dos autovalores
    pares_autovalores_autovetores = [(np.abs(autovalores[i]), autovetores[:, i]) for i in range(len(autovalores))]
    pares_autovalores_autovetores.sort(key=lambda x: x[0], reverse=True)

    # Seleção dos componentes principais com base no número especificado
    componentes = np.hstack([pares_autovalores_autovetores[i][1].reshape(-1, 1) for i in range(num_componentes)])

    # Transformação dos dados originais usando os componentes principais
    dados_transformados = np.dot(dados, componentes)

    return dados_transformados

def kmeans(dados, k, max_iteracoes=10):
    melhor_swc = None
    melhores_clusters = None

    distancias = cdist(dados, dados, metric='euclidean')

    for _ in range(max_iteracoes):
        np.random.seed()
        indices_centroides = np.random.choice(range(len(dados)), size=k, replace=False)
        centroides = dados[indices_centroides]
        clusters_anteriores = None

        while True:
            clusters = np.argmin(distancias[:, indices_centroides], axis=1)

            if np.array_equal(clusters, clusters_anteriores):
                break

            centroides = np.array([np.mean(dados[clusters == cluster], axis=0) for cluster in np.arange(k)])

            clusters_anteriores = clusters

        swc = calcular_swc(dados, clusters)
        if melhor_swc is None or swc > melhor_swc:
            melhor_swc = swc
            melhores_clusters = clusters

    return melhores_clusters, melhor_swc

dados = df_norm.copy()

num_componentes_lista = [1, 2, 3, 4,5,6,7]  # Testar diferentes números de componentes
valores_k = [2, 3, 4, 5]  # Testar diferentes valores de K

melhor_swc = None
melhores_clusters = None
melhor_num_componentes = None
melhor_k = None

for num_componentes in num_componentes_lista:
    dados_transformados = pca(df_norm.values, num_componentes)

    for k in valores_k:
        clusters, swc = kmeans(dados_transformados, k)

        if melhor_swc is None or swc > melhor_swc:
            melhor_swc = swc
            melhores_clusters = clusters
            melhor_num_componentes = num_componentes
            melhor_k = k

print("Melhor número de componentes do PCA:", melhor_num_componentes)
print("Melhor valor de K:", melhor_k)
print("SWC:", melhor_swc)

Melhor número de componentes do PCA: 1
Melhor valor de K: 2
SWC: 0.5620055924415026
